In [2]:
import cv2
import dlib
import os
import sys
import random
# 存储位置
output_dir = 'E:/Ai/face/face/'
size = 256
 
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
# 改变图片的亮度与对比度
 
def relight(img, light=1, bias=0):
    w = img.shape[1]
    h = img.shape[0]
    #image = []
    for i in range(0,w):
        for j in range(0,h):
            for c in range(3):
                tmp = int(img[j,i,c]*light + bias)
                if tmp > 255:
                    tmp = 255
                elif tmp < 0:
                    tmp = 0
                img[j,i,c] = tmp
    return img
 
#使用dlib自带的frontal_face_detector作为我们的特征提取器
detector = dlib.get_frontal_face_detector()
# 打开摄像头 参数为输入流，可以为摄像头或视频文件
camera = cv2.VideoCapture(0)
#camera = cv2.VideoCapture('C:/Users/CUNGU/Videos/Captures/wang.mp4')

index = 1
while True:
    if (index <= 20):#存储15张人脸特征图像
        print('Being processed picture %s' % index)
        # 从摄像头读取照片
        success, img = camera.read()
        # 转为灰度图片
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        # 使用detector进行人脸检测
        dets = detector(gray_img, 1)
 
        for i, d in enumerate(dets):
            x1 = d.top() if d.top() > 0 else 0
            y1 = d.bottom() if d.bottom() > 0 else 0
            x2 = d.left() if d.left() > 0 else 0
            y2 = d.right() if d.right() > 0 else 0
 
            face = img[x1:y1,x2:y2]
            # 调整图片的对比度与亮度， 对比度与亮度值都取随机数，这样能增加样本的多样性
            face = relight(face, random.uniform(0.5, 1.5), random.randint(-50, 50))
 
            face = cv2.resize(face, (size,size))
 
            cv2.imshow('image', face)
 
            cv2.imwrite(output_dir+'/'+str(index)+'.jpg', face)
 
            index += 1
        key = cv2.waitKey(30) & 0xff
        if key == 27:
            break
    else:
        print('Finished!')
        # 释放摄像头 release camera
        camera.release()
        # 删除建立的窗口 delete all the windows
        cv2.destroyAllWindows()
        break


Being processed picture 1
Being processed picture 1
Being processed picture 2
Being processed picture 3
Being processed picture 4
Being processed picture 4
Being processed picture 4
Being processed picture 4
Being processed picture 4
Being processed picture 4
Being processed picture 4
Being processed picture 4
Being processed picture 4
Being processed picture 4
Being processed picture 4
Being processed picture 4
Being processed picture 4
Being processed picture 4
Being processed picture 4
Being processed picture 4
Being processed picture 4
Being processed picture 4
Being processed picture 5
Being processed picture 6
Being processed picture 7
Being processed picture 8
Being processed picture 8
Being processed picture 8
Being processed picture 8
Being processed picture 8
Being processed picture 8
Being processed picture 8
Being processed picture 8
Being processed picture 8
Being processed picture 8
Being processed picture 8
Being processed picture 8
Being processed picture 8
Being proces

In [12]:
from cv2 import cv2 as cv2
import os
import dlib
from skimage import io
import csv
import numpy as np

# 要读取人脸图像文件的路径
path_images_from_camera = "E:/Ai/face/face/"

# Dlib 正向人脸检测器
detector = dlib.get_frontal_face_detector()

# Dlib 人脸预测器
predictor = dlib.shape_predictor("E:/Ai/face/shape_predictor_68_face_landmarks.dat")

# Dlib 人脸识别模型
# Face recognition model, the object maps human faces into 128D vectors
face_rec = dlib.face_recognition_model_v1("E:/Ai/face/dlib_face_recognition_resnet_model_v1.dat")


# 返回单张图像的 128D 特征
def return_128d_features(path_img):
    img_rd = io.imread(path_img)
    img_gray = cv2.cvtColor(img_rd, cv2.COLOR_BGR2RGB)
    faces = detector(img_gray, 1)

    print("%-40s %-20s" % ("检测到人脸的图像 / image with faces detected:", path_img), '\n')

    # 因为有可能截下来的人脸再去检测，检测不出来人脸了
    # 所以要确保是 检测到人脸的人脸图像 拿去算特征
    if len(faces) != 0:
        shape = predictor(img_gray, faces[0])
        face_descriptor = face_rec.compute_face_descriptor(img_gray, shape)
    else:
        face_descriptor = 0
        print("no face")

    return face_descriptor


# 将文件夹中照片特征提取出来, 写入 CSV
def return_features_mean_personX(path_faces_personX):
    features_list_personX = []
    photos_list = os.listdir(path_faces_personX)
    if photos_list:
        for i in range(len(photos_list)):
            # 调用return_128d_features()得到128d特征
            print("%-40s %-20s" % ("正在读的人脸图像 / image to read:", path_faces_personX + "/" + photos_list[i]))
            features_128d = return_128d_features(path_faces_personX + "/" + photos_list[i])
            #  print(features_128d)
            # 遇到没有检测出人脸的图片跳过
            if features_128d == 0:
                i += 1
            else:
                features_list_personX.append(features_128d)
    else:
        print("文件夹内图像文件为空 / Warning: No images in " + path_faces_personX + '/', '\n')

    # 计算 128D 特征的均值
    # N x 128D -> 1 x 128D
    if features_list_personX:
        features_mean_personX = np.array(features_list_personX).mean(axis=0)
    else:
        features_mean_personX = '0'

    return features_mean_personX


# 读取某人所有的人脸图像的数据
people = os.listdir(path_images_from_camera)
people.sort()

with open("E:/Ai/face/features2_all.csv", "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    for person in people:
        print("##### " + person + " #####")
        # Get the mean/average features of face/personX, it will be a list with a length of 128D
        features_mean_personX = return_features_mean_personX(path_images_from_camera + person)
        writer.writerow(features_mean_personX)
        print("特征均值 / The mean of features:", list(features_mean_personX))
        print('\n')
    print("所有录入人脸数据存入 / Save all the features of faces registered into: D:/myworkspace/JupyterNotebook/People/feature/features_all2.csv")


##### f1 #####
正在读的人脸图像 / image to read:                E:/Ai/face/face/f1/1.jpg
检测到人脸的图像 / image with faces detected:    E:/Ai/face/face/f1/1.jpg 

正在读的人脸图像 / image to read:                E:/Ai/face/face/f1/10.jpg
检测到人脸的图像 / image with faces detected:    E:/Ai/face/face/f1/10.jpg 

正在读的人脸图像 / image to read:                E:/Ai/face/face/f1/11.jpg
检测到人脸的图像 / image with faces detected:    E:/Ai/face/face/f1/11.jpg 

正在读的人脸图像 / image to read:                E:/Ai/face/face/f1/12.jpg
检测到人脸的图像 / image with faces detected:    E:/Ai/face/face/f1/12.jpg 

正在读的人脸图像 / image to read:                E:/Ai/face/face/f1/13.jpg
检测到人脸的图像 / image with faces detected:    E:/Ai/face/face/f1/13.jpg 

正在读的人脸图像 / image to read:                E:/Ai/face/face/f1/14.jpg
检测到人脸的图像 / image with faces detected:    E:/Ai/face/face/f1/14.jpg 

正在读的人脸图像 / image to read:                E:/Ai/face/face/f1/15.jpg
检测到人脸的图像 / image with faces detected:    E:/Ai/face/face/f1/15.jpg 

正在读的人脸图像 / image to read:          

正在读的人脸图像 / image to read:                E:/Ai/face/face/zhangzhitong/8.jpg
检测到人脸的图像 / image with faces detected:    E:/Ai/face/face/zhangzhitong/8.jpg 

正在读的人脸图像 / image to read:                E:/Ai/face/face/zhangzhitong/9.jpg
检测到人脸的图像 / image with faces detected:    E:/Ai/face/face/zhangzhitong/9.jpg 

特征均值 / The mean of features: [-0.06223821649327874, 0.08428077474236488, 0.020015016850084066, -0.045205201208591464, -0.0536767240613699, -0.09202293679118156, -0.06285716146230698, -0.1003984309732914, 0.13183749802410602, -0.09331661127507687, 0.20091252624988556, -0.03501697913743555, -0.20412082374095916, -0.140727748721838, -0.00218303226865828, 0.1593516491353512, -0.1514843013137579, -0.1285684559494257, -0.03329265913926065, 0.009692828822880983, 0.072564204223454, -0.019228952174307777, 0.06759697068482637, 0.021031528478488326, -0.03364702556282282, -0.3749030277132988, -0.14697603955864907, -0.11117860600352288, 0.0020200711209326983, -0.06356278331950307, -0.085827862843

In [1]:
# 摄像头实时人脸识别
import os
import winsound # 系统音效
import dlib          # 人脸处理的库 Dlib
import csv # 存入表格
import time
import sys
import numpy as np   # 数据处理的库 numpy
from cv2 import cv2 as cv2           # 图像处理的库 OpenCv
import pandas as pd  # 数据处理的库 Pandas


# 人脸识别模型，提取128D的特征矢量
# face recognition model, the object maps human faces into 128D vectors
# Refer this tutorial: http://dlib.net/python/index.html#dlib.face_recognition_model_v1
facerec = dlib.face_recognition_model_v1("E:/Ai/face/dlib_face_recognition_resnet_model_v1.dat")


# 计算两个128D向量间的欧式距离
# compute the e-distance between two 128D features
def return_euclidean_distance(feature_1, feature_2):
    feature_1 = np.array(feature_1)
    feature_2 = np.array(feature_2)
    dist = np.sqrt(np.sum(np.square(feature_1 - feature_2)))
    return dist


# 处理存放所有人脸特征的 csv
path_features_known_csv = "E:/Ai/face/features2_all.csv"
csv_rd = pd.read_csv(path_features_known_csv, header=None)


# 用来存放所有录入人脸特征的数组
# the array to save the features of faces in the database
features_known_arr = []

# 读取已知人脸数据
# print known faces
for i in range(csv_rd.shape[0]):
    features_someone_arr = []
    for j in range(0, len(csv_rd.iloc[i, :])):
        features_someone_arr.append(csv_rd.iloc[i, :][j])
    features_known_arr.append(features_someone_arr)
print("Faces in Database：", len(features_known_arr))

# Dlib 检测器和预测器
# The detector and predictor will be used
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('E:/Ai/face/shape_predictor_68_face_landmarks.dat')

# 创建 cv2 摄像头对象
# cv2.VideoCapture(0) to use the default camera of PC,
# and you can use local video name by use cv2.VideoCapture(filename)
cap = cv2.VideoCapture(0)

# cap.set(propId, value)
# 设置视频参数，propId 设置的视频参数，value 设置的参数值
cap.set(3, 480)

# cap.isOpened() 返回 true/false 检查初始化是否成功
# when the camera is open
while cap.isOpened():

    flag, img_rd = cap.read()
    kk = cv2.waitKey(1)

    # 取灰度
    img_gray = cv2.cvtColor(img_rd, cv2.COLOR_RGB2GRAY)

    # 人脸数 faces
    faces = detector(img_gray, 0)

    # 待会要写的字体 font to write later
    font = cv2.FONT_HERSHEY_COMPLEX

    # 存储当前摄像头中捕获到的所有人脸的坐标/名字
    # the list to save the positions and names of current faces captured
    pos_namelist = []
    name_namelist = []

    # 按下 q 键退出
    # press 'q' to exit
    if kk == ord('q'):
        break
    else:
        # 检测到人脸 when face detected
        if len(faces) != 0:  
            # 获取当前捕获到的图像的所有人脸的特征，存储到 features_cap_arr
            # get the features captured and save into features_cap_arr
            features_cap_arr = []
            for i in range(len(faces)):
                shape = predictor(img_rd, faces[i])
                features_cap_arr.append(facerec.compute_face_descriptor(img_rd, shape))

            # 遍历捕获到的图像中所有的人脸
            # traversal all the faces in the database
            for k in range(len(faces)):
                print("##### camera person", k+1, "#####")
                # 让人名跟随在矩形框的下方
                # 确定人名的位置坐标
                # 先默认所有人不认识，是 unknown
                # set the default names of faces with "unknown"
                name_namelist.append("unknown")

                # 每个捕获人脸的名字坐标 the positions of faces captured
                pos_namelist.append(tuple([faces[k].left(), int(faces[k].bottom() + (faces[k].bottom() - faces[k].top())/4)]))

                # 对于某张人脸，遍历所有存储的人脸特征
                # for every faces detected, compare the faces in the database
                e_distance_list = []
                for i in range(len(features_known_arr)):
                    # 如果 person_X 数据不为空
                    if str(features_known_arr[i][0]) != '0.0':
                        print("with person", str(i + 1), "the e distance: ", end='')
                        e_distance_tmp = return_euclidean_distance(features_cap_arr[k], features_known_arr[i])
                        print(e_distance_tmp)
                        e_distance_list.append(e_distance_tmp)
                    else:
                        # 空数据 person_X
                        e_distance_list.append(999999999)
                # 找出最接近的一个人脸数据是第几个
                # Find the one with minimum e distance
                similar_person_num = e_distance_list.index(min(e_distance_list))
                print("Minimum e distance with person", int(similar_person_num)+1)
                
                # 计算人脸识别特征与数据集特征的欧氏距离
                # 距离小于0.4则标出为可识别人物
                if min(e_distance_list) < 0.4:
                    # 这里可以修改摄像头中标出的人名
                    # Here you can modify the names shown on the camera
                    # 1、遍历文件夹目录
                    folder_name = 'E:/Ai/face/face/'
                    # 最接近的人脸
                    sum=similar_person_num+1
                    key_id=1 # 从第一个人脸数据文件夹进行对比
                    # 获取文件夹中的文件名:LQH、YYQX、WY、WL...
                    file_names = os.listdir(folder_name)
                    for name in file_names:
                        # print(name+'->'+str(key_id))
                        if sum ==key_id:
                            #winsound.Beep(300,500)# 响铃：300频率，500持续时间
                            name_namelist[k] = name[0:]#人名删去第一个数字（用于视频输出标识）
                        key_id += 1
                    # 播放欢迎光临音效
                    #playsound('D:/myworkspace/JupyterNotebook/People/music/welcome.wav')
                    # print("May be person "+str(int(similar_person_num)+1))
                    # -----------筛选出人脸并保存到visitor文件夹------------
                    for i, d in enumerate(faces):
                        x1 = d.top() if d.top() > 0 else 0
                        y1 = d.bottom() if d.bottom() > 0 else 0
                        x2 = d.left() if d.left() > 0 else 0
                        y2 = d.right() if d.right() > 0 else 0
                        face = img_rd[x1:y1,x2:y2]
                        size = 64
                        face = cv2.resize(face, (size,size))
                        # 要存储visitor人脸图像文件的路径
                        path_visitors_save_dir = "E:/Ai/face/PersonRecognitionDlib/visitor/known/"
                        # 存储格式：2019-06-24-14-33-40LQH.jpg
                        now_time = time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())
                        save_name = str(now_time)+str(name_namelist[k])+'.jpg'
                        # print(save_name)
                        # 本次图片保存的完整url
                        save_path = path_visitors_save_dir+'/'+ save_name    
                        # 遍历visitor文件夹所有文件名
                        visitor_names = os.listdir(path_visitors_save_dir)
                        visitor_name=''
                        for name in visitor_names:
                            # 名字切片到分钟数：2019-06-26-11-33-00LQH.jpg
                            visitor_name=(name[0:16]+'-00'+name[19:])
                        # print(visitor_name)
                        visitor_save=(save_name[0:16]+'-00'+save_name[19:])
                        # print(visitor_save)
                        # 一分钟之内重复的人名不保存
                        if visitor_save!=visitor_name:
                            cv2.imwrite(save_path, face)
                            print('新存储：'+path_visitors_save_dir+'/'+str(now_time)+str(name_namelist[k])+'.jpg')
                        else:
                            print('重复，未保存！')
                            
                else:
                    # 播放无法识别音效
                    #playsound('D:/myworkspace/JupyterNotebook/People/music/sorry.wav')
                    print("Unknown person")
                    # -----保存图片-------
                    # -----------筛选出人脸并保存到visitor文件夹------------
                    for i, d in enumerate(faces):
                        x1 = d.top() if d.top() > 0 else 0
                        y1 = d.bottom() if d.bottom() > 0 else 0
                        x2 = d.left() if d.left() > 0 else 0
                        y2 = d.right() if d.right() > 0 else 0
                        face = img_rd[x1:y1,x2:y2]
                        size = 64
                        face = cv2.resize(face, (size,size))
                        # 要存储visitor-》unknown人脸图像文件的路径
                        path_visitors_save_dir = "E:/Ai/face/PersonRecognitionDlib/visitor/unknown/"
                        # 存储格式：2019-06-24-14-33-40unknown.jpg
                        now_time = time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())
                        # print(save_name)
                        # 本次图片保存的完整url
                        save_path = path_visitors_save_dir+'/'+ str(now_time)+'unknown.jpg'
                        cv2.imwrite(save_path, face)
                        print('新存储：'+path_visitors_save_dir+'/'+str(now_time)+'unknown.jpg')
                
                # 矩形框
                # draw rectangle
                for kk, d in enumerate(faces):
                    # 绘制矩形框
                    cv2.rectangle(img_rd, tuple([d.left(), d.top()]), tuple([d.right(), d.bottom()]), (0, 255, 255), 2)
                print('\n')

            # 在人脸框下面写人脸名字
            # write names under rectangle
            for i in range(len(faces)):
                cv2.putText(img_rd, name_namelist[i], pos_namelist[i], font, 0.8, (0, 255, 255), 1, cv2.LINE_AA)

    print("Faces in camera now:", name_namelist, "\n")

    #cv2.putText(img_rd, "Press 'q': Quit", (20, 450), font, 0.8, (84, 255, 159), 1, cv2.LINE_AA)
    cv2.putText(img_rd, "Face Recognition", (20, 40), font, 1, (0, 0, 255), 1, cv2.LINE_AA)
    cv2.putText(img_rd, "Visitors: " + str(len(faces)), (20, 100), font, 1, (0, 0, 255), 1, cv2.LINE_AA)

    # 窗口显示 show with opencv
    cv2.imshow("camera", img_rd)

# 释放摄像头 release camera
cap.release()

# 删除建立的窗口 delete all the windows
cv2.destroyAllWindows()


Faces in Database： 2
Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera n

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in cam

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in cam

##### camera person 1 #####
with person 1 the e distance: 0.2897992010949741
with person 2 the e distance: 0.4681970266318899
Minimum e distance with person 1
重复，未保存！


Faces in camera now: ['f1'] 

##### camera person 1 #####
with person 1 the e distance: 0.2766298338825878
with person 2 the e distance: 0.43775992271743575
Minimum e distance with person 1
重复，未保存！


Faces in camera now: ['f1'] 

##### camera person 1 #####
with person 1 the e distance: 0.27489160824659037
with person 2 the e distance: 0.44461145422548326
Minimum e distance with person 1
重复，未保存！


Faces in camera now: ['f1'] 

##### camera person 1 #####
with person 1 the e distance: 0.26502404903815147
with person 2 the e distance: 0.4142433082772051
Minimum e distance with person 1
重复，未保存！


Faces in camera now: ['f1'] 

##### camera person 1 #####
with person 1 the e distance: 0.2828559979003156
with person 2 the e distance: 0.4254968149147873
Minimum e distance with person 1
重复，未保存！


Faces in camera now: ['f1'] 

#

##### camera person 1 #####
with person 1 the e distance: 0.29985636237386143
with person 2 the e distance: 0.4619650906334429
Minimum e distance with person 1
重复，未保存！


Faces in camera now: ['f1'] 

##### camera person 1 #####
with person 1 the e distance: 0.3218601850055212
with person 2 the e distance: 0.4850045306421789
Minimum e distance with person 1
重复，未保存！


Faces in camera now: ['f1'] 

##### camera person 1 #####
with person 1 the e distance: 0.313174861702214
with person 2 the e distance: 0.4875978619480735
Minimum e distance with person 1
重复，未保存！


Faces in camera now: ['f1'] 

Faces in camera now: [] 

##### camera person 1 #####
with person 1 the e distance: 0.29828391847834856
with person 2 the e distance: 0.503080157996611
Minimum e distance with person 1
重复，未保存！


Faces in camera now: ['f1'] 

##### camera person 1 #####
with person 1 the e distance: 0.2658415752958962
with person 2 the e distance: 0.4245921646319732
Minimum e distance with person 1
重复，未保存！


Faces in 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

##### camera person 1 #####
with person 1 the e distance: 0.36295711847304646
with person 2 the e distance: 0.3992840302138724
Minimum e distance with person 1
重复，未保存！


Faces in camera now: ['f1'] 

Faces in camera now: [] 

Faces in camera now: [] 

##### camera person 1 #####
with person 1 the e distance: 0.323286975512

##### camera person 1 #####
with person 1 the e distance: 0.3112631807691
with person 2 the e distance: 0.41361732409257795
Minimum e distance with person 1
重复，未保存！


Faces in camera now: ['f1'] 

##### camera person 1 #####
with person 1 the e distance: 0.34873370148608845
with person 2 the e distance: 0.4409526204704826
Minimum e distance with person 1
重复，未保存！


Faces in camera now: ['f1'] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces

##### camera person 1 #####
with person 1 the e distance: 0.28107876001301507
with person 2 the e distance: 0.43243045598980895
Minimum e distance with person 1
重复，未保存！


Faces in camera now: ['f1'] 

Faces in camera now: [] 

##### camera person 1 #####
with person 1 the e distance: 0.2854464901855411
with person 2 the e distance: 0.42254191796156576
Minimum e distance with person 1
重复，未保存！


Faces in camera now: ['f1'] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

F